In [1]:
# clone darknet repo
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 14240, done.
remote: Total 14240 (delta 0), reused 0 (delta 0), pack-reused 14240
Receiving objects: 100% (14240/14240), 12.79 MiB | 17.55 MiB/s, done.
Resolving deltas: 100% (9677/9677), done.


In [2]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

/content/darknet


In [3]:
# verify CUDA
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [4]:
# make darknet (builds darknet so that you can then use the darknet executable file to run or train object detectors)
!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -DCUDNN_HALF -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -DCUDNN_HALF -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
  

In [5]:
%cd ..
!git clone https://github.com/theAIGuysCode/tensorflow-yolov4-tflite.git

/content
Cloning into 'tensorflow-yolov4-tflite'...
remote: Enumerating objects: 907, done.
remote: Total 907 (delta 0), reused 0 (delta 0), pack-reused 907
Receiving objects: 100% (907/907), 211.45 MiB | 25.37 MiB/s, done.
Resolving deltas: 100% (438/438), done.


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

In [8]:
%cd tensorflow-yolov4-tflite/
!pip install -r requirements-gpu.txt

/content/tensorflow-yolov4-tflite
     |████████████████████████████████| 320.3MB 55kB/s 
     |████████████████████████████████| 28.7MB 110kB/s 
     |████████████████████████████████| 3.0MB 31.3MB/s 
     |████████████████████████████████| 460kB 15.3MB/s 
ERROR: tensorflow 2.3.0 has requirement tensorboard<3,>=2.3.0, but you'll have tensorboard 2.2.2 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30


Note: make sure to change path of __C.YOLO.CLASSES in /core/config.py!

In [ ]:
!python save_model.py --weights /mydrive/yolov4/backup/yolov4-custom_best.weights --output /mydrive/yolov4/squirrel --input_size 416 --model yolov4

In [ ]:
!python detect.py --weights /mydrive/yolov4/squirrel --size 416 --model yolov4 --images /mydrive/yolov4/squirrel.jpeg --output /mydrive/yolov4/detections/

In [ ]:
!python detect_video.py --weights /mydrive/yolov4/squirrel --size 416 --model yolov4 --video /mydrive/yolov4/squirtl_demo.mov --output /mydrive/yolov4/detections/results.avi

In [15]:
!python save_model.py --weights /mydrive/yolov4/backup/yolov4-custom_best.weights --output /mydrive/yolov4/squirrel_lite/ --input_size 416 --model yolov4 --framework tflite

2020-08-14 15:34:08.212905: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-08-14 15:34:10.267443: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-08-14 15:34:10.287463: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-08-14 15:34:10.288417: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla K80 computeCapability: 3.7
coreClock: 0.8235GHz coreCount: 13 deviceMemorySize: 11.17GiB deviceMemoryBandwidth: 223.96GiB/s
2020-08-14 15:34:10.288517: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-08-14 15:34:10.290991: I tensorflow/stream_executor/platform/defa

In [16]:
!python convert_tflite.py --weights /mydrive/yolov4/squirrel_lite --output /mydrive/yolov4/squirrel_lite.tflite

2020-08-14 15:38:29.874867: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-08-14 15:38:31.760659: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-08-14 15:38:31.779089: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-08-14 15:38:31.779898: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla K80 computeCapability: 3.7
coreClock: 0.8235GHz coreCount: 13 deviceMemorySize: 11.17GiB deviceMemoryBandwidth: 223.96GiB/s
2020-08-14 15:38:31.779957: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-08-14 15:38:31.782551: I tensorflow/stream_executor/platform/defa

In [17]:
!python convert_tflite.py --weights /mydrive/yolov4/squirrel_lite --output /mydrive/yolov4/squirrel_lite-fp16.tflite --quantize_mode float16

2020-08-14 15:39:41.082746: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-08-14 15:39:43.043434: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-08-14 15:39:43.065537: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-08-14 15:39:43.066498: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla K80 computeCapability: 3.7
coreClock: 0.8235GHz coreCount: 13 deviceMemorySize: 11.17GiB deviceMemoryBandwidth: 223.96GiB/s
2020-08-14 15:39:43.066577: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-08-14 15:39:43.069968: I tensorflow/stream_executor/platform/defa

In [26]:
!python detect.py --weights /mydrive/yolov4/squirrel_lite/squirrel_lite.tflite --size 416 --model yolov4 --images squirrel.jpeg --framework tflite --output /mydrive/yolov4/detections/output.png

2020-08-14 15:50:35.129956: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-08-14 15:50:37.205063: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-08-14 15:50:37.223282: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-08-14 15:50:37.224156: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla K80 computeCapability: 3.7
coreClock: 0.8235GHz coreCount: 13 deviceMemorySize: 11.17GiB deviceMemoryBandwidth: 223.96GiB/s
2020-08-14 15:50:37.224217: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-08-14 15:50:37.226853: I tensorflow/stream_executor/platform/defa

In [27]:
!python detect.py --weights /mydrive/yolov4/squirrel --size 416 --model yolov4 --images /mydrive/yolov4/squirrel.jpeg --output /mydrive/yolov4/detections/

2020-08-14 15:51:37.858652: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-08-14 15:51:39.696035: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-08-14 15:51:39.714332: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-08-14 15:51:39.715154: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla K80 computeCapability: 3.7
coreClock: 0.8235GHz coreCount: 13 deviceMemorySize: 11.17GiB deviceMemoryBandwidth: 223.96GiB/s
2020-08-14 15:51:39.715218: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-08-14 15:51:39.717516: I tensorflow/stream_executor/platform/defa

In [28]:
!python detect.py --weights /mydrive/yolov4/squirrel_lite/squirrel_lite-fp16.tflite --size 416 --model yolov4 --framework tflite --images /mydrive/yolov4/squirrel.jpeg --output /mydrive/yolov4/detections/tflite-fps16_

2020-08-14 15:55:42.231917: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-08-14 15:55:44.181346: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-08-14 15:55:44.203526: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-08-14 15:55:44.204344: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla K80 computeCapability: 3.7
coreClock: 0.8235GHz coreCount: 13 deviceMemorySize: 11.17GiB deviceMemoryBandwidth: 223.96GiB/s
2020-08-14 15:55:44.204395: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-08-14 15:55:44.206619: I tensorflow/stream_executor/platform/defa

In [32]:
!python save_model.py --weights /mydrive/yolov4/backup/yolov4-custom_best.weights --output /mydrive/yolov4/squirel_trt/squirrel.tf --input_size 416 --model yolov4 --framework trt

2020-08-14 16:20:13.775929: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-08-14 16:20:15.408523: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-08-14 16:20:15.427037: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-08-14 16:20:15.427773: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla K80 computeCapability: 3.7
coreClock: 0.8235GHz coreCount: 13 deviceMemorySize: 11.17GiB deviceMemoryBandwidth: 223.96GiB/s
2020-08-14 16:20:15.427830: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-08-14 16:20:15.429988: I tensorflow/stream_executor/platform/defa